# Ball to Ball DataSet

Columns

1. start_date
2. end_date
3. city
4. venue
5. event_name
6. match_type
7. match_number
8. team_type
9. team_1
10. team_2
11. toss_winner
12. toss_decision
13. innings
14. overs
15. batting_team
16. bowling_team
17. striker
18. non_striker
19. bowler
20. singles
21. 4s
22. 6s
23. legbyes
24. byes
25. wides
26. no_balls
27. total_runs
28. wickets
29. wicket_type
30. wicket_taken_by
31. player_disissmied

In [1]:
import pandas as pd
import numpy as np
import json
import glob
import decimal
import os
pd.set_option('display.max_columns', None)

In [2]:
start_date = [] # $
end_date = [] # $
city = [] # $
venue = [] # $
event_name = [] # $
match_type = [] # $
match_number = [] # $
team_type = [] # $
team_1 = [] # $
team_2 = [] # $
toss_winner = [] # $
toss_decision = [] #| $
innings = [] # $
batting_team = [] # $
bowling_team = [] # $
overs = [] # $
striker = [] # $
non_striker = [] # $
bowler = [] # $
singles = [] # $
s4 = [] #  $
s6 = [] # $
legbyes = [] # $
byes = [] # $
wides = [] # $
no_balls = [] # $
total_runs = [] # $
wickets = [] # $
wicket_type = [] # $
wicket_taken_by = [] # $
player_disissmied = [] # $

In [3]:
def get_striker_bowler_non_striker(data):
    striker.append(ball['batter'])
    bowler.append(ball['bowler'])
    non_striker.append(ball['non_striker'])

In [4]:
def get_innings_balls(inning, data):
    j = 0
    
    team1 = data['info']['teams'][0]
    team2 = data['info']['teams'][1]

    for over in data['innings'][inning]['overs']:
        batTeam = data['innings'][inning]['team']
        end = len(over['deliveries']) + 1
        innings.extend([inning + 1] * len(over['deliveries']))
        balls = np.array(list(range(1, end)))
        overs.extend((balls + j)/10)
        if batTeam == team1:
            batting_team.extend([team1] * len(over['deliveries']))
            bowling_team.extend([team2] * len(over['deliveries']))
        if batTeam == team2:
            batting_team.extend([team2] * len(over['deliveries']))
            bowling_team.extend([team1] * len(over['deliveries']))
        j = j + 10

In [5]:
def get_runs(ball):
    total_runs.append(ball['runs']['total']) 
    if ball['runs']['batter'] <= 3:
        singles.append(ball['runs']['batter'])
    else:
        singles.append(0)

    if ball['runs']['batter'] == 4:
        s4.append(1)
    else:
        s4.append(0)

    if ball['runs']['batter'] == 6:
        s6.append(1)
    else:
        s6.append(0)

In [6]:
def total_wickets(ball):

    # Checking if wicket is taken or not in this over
    li_wk = ball.keys()
    if 'wickets' in li_wk:
        wickets.append(1)
        player_disissmied.append(ball['wickets'][0]['player_out'])
        kind = ball['wickets'][0]['kind']
        wicket_type.append(kind)

        if kind == 'caught' or kind == 'run out' or kind == 'stumped':
            fielder = ball['wickets'][0]
            if 'fielders' in fielder:
                name = ball['wickets'][0]['fielders'][0].keys()
                if 'name' in name:
                    fielder_name = ball['wickets'][0]['fielders'][0]['name']
                    wicket_taken_by.append(fielder_name)
                else:
                    wicket_taken_by.append('missing')
            else:
                wicket_taken_by.append('missing')       
        else:
            wicket_taken_by.append(ball['bowler'])
    else:
        wickets.append(0)
        player_disissmied.append('No Name')
        wicket_type.append('Not Out')
        wicket_taken_by.append('No Name')


In [7]:
def get_extra(ball):

    extra_info = ball.keys()
    if 'extras' in extra_info:
        li = ball['extras'].keys()
        if 'noballs'in li:
            no_balls.append(ball['extras']['noballs'])
        else:
            no_balls.append(0)
        if 'wides'in li:
            wides.append(ball['extras']['wides'])
        else:
            wides.append(0)
        if 'legbyes' in li:
            legbyes.append(ball['extras']['legbyes'])
        else:
            legbyes.append(0)
        if 'byes' in li:
            byes.append(ball['extras']['byes'])
        else:
            byes.append(0)
    else:
        no_balls.append(0)
        wides.append(0)
        legbyes.append(0)
        byes.append(0)

In [8]:
def get_comm(data):

    team1 = data['info']['teams'][0]
    team2 = data['info']['teams'][1]
    toss_win = data['info']['toss']['winner']
    decision = data['info']['toss']['decision']
    info = data['info'].keys()
    # match start Date
    start_date.append(data['info']['dates'][0])

    # match end Date
    end_date.append(data['info']['dates'][-1])

    # Match Played in City
    if 'city' in info:
        city.append(data['info']['city'])
    else:
        city.append('missing')

    # Stadium name
    if 'venue' in info:
        venue.append(data['info']['venue'])
    else:
        venue.append('missing')

    # Event Name
    if 'event' in info:
        event_name.append(data['info']['event']['name'])
    else:
        event_name.append('missing')
        
    # Match type (Test, ODI, T20, ICC)
    if 'match_type' in info:
        match_type.append(data['info']['match_type'])
    else:
        match_type.append('missing')
        
    # Match Number
    if 'match_type_number' in info:
        match_number.append(data['info']['match_type_number'])
    else:
        match_number.append('missing')  

    # Team Type (Internation, Domestic)
    if 'team_type' in info:
        team_type.append(data['info']['team_type'])
    else:
        team_type.append('missing')

    # Team 1 & Team 2
    team_1.append(team1)
    team_2.append(team2)

    # Toss winner and Decision
    toss_winner.append(toss_win)
    toss_decision.append(decision)

In [9]:
files = []

for file in os.listdir("Data/2022_female_json/"):
    if file.endswith(".json"):
        files.append(os.path.join("Data/2022_female_json/", file))

for file in files:
    with open(file) as f: 
        data = json.load(f)
        print(f'fileName: {file}')
        
        for inn in range(0, len(data['innings'])):          
            i = 0
            ove = data['innings'][inn].keys()
            if 'overs' in ove:
                get_innings_balls(inn, data)
                for over in data['innings'][inn]['overs']:                         
                    for ball in data['innings'][inn]['overs'][i]['deliveries']:

                        get_striker_bowler_non_striker(data)

                        get_runs(ball)

                        total_wickets(ball)

                        get_extra(ball)

                        get_comm(data)
                    i = i + 1    

fileName: Data/2022_female_json/1243908.json
fileName: Data/2022_female_json/1243909.json
fileName: Data/2022_female_json/1243910.json
fileName: Data/2022_female_json/1243911.json
fileName: Data/2022_female_json/1243912.json
fileName: Data/2022_female_json/1243913.json
fileName: Data/2022_female_json/1243914.json
fileName: Data/2022_female_json/1243915.json
fileName: Data/2022_female_json/1243916.json
fileName: Data/2022_female_json/1243917.json
fileName: Data/2022_female_json/1243918.json
fileName: Data/2022_female_json/1243919.json
fileName: Data/2022_female_json/1243920.json
fileName: Data/2022_female_json/1243921.json
fileName: Data/2022_female_json/1243922.json
fileName: Data/2022_female_json/1243923.json
fileName: Data/2022_female_json/1243924.json
fileName: Data/2022_female_json/1289031.json
fileName: Data/2022_female_json/1289032.json
fileName: Data/2022_female_json/1289033.json
fileName: Data/2022_female_json/1289034.json
fileName: Data/2022_female_json/1289035.json
fileName: 

fileName: Data/2022_female_json/1322341.json
fileName: Data/2022_female_json/1322342.json
fileName: Data/2022_female_json/1322343.json
fileName: Data/2022_female_json/1323178.json
fileName: Data/2022_female_json/1323179.json
fileName: Data/2022_female_json/1323180.json
fileName: Data/2022_female_json/1323181.json
fileName: Data/2022_female_json/1323554.json
fileName: Data/2022_female_json/1323555.json
fileName: Data/2022_female_json/1323556.json
fileName: Data/2022_female_json/1297927.json
fileName: Data/2022_female_json/1297928.json
fileName: Data/2022_female_json/1297929.json
fileName: Data/2022_female_json/1297930.json
fileName: Data/2022_female_json/1297931.json
fileName: Data/2022_female_json/1297932.json
fileName: Data/2022_female_json/1297933.json
fileName: Data/2022_female_json/1297934.json
fileName: Data/2022_female_json/1297935.json
fileName: Data/2022_female_json/1297936.json
fileName: Data/2022_female_json/1297937.json
fileName: Data/2022_female_json/1297938.json
fileName: 

fileName: Data/2022_female_json/1323599.json
fileName: Data/2022_female_json/1323600.json
fileName: Data/2022_female_json/1323601.json
fileName: Data/2022_female_json/1323602.json
fileName: Data/2022_female_json/1323603.json
fileName: Data/2022_female_json/1323604.json
fileName: Data/2022_female_json/1323605.json
fileName: Data/2022_female_json/1323606.json
fileName: Data/2022_female_json/1323607.json
fileName: Data/2022_female_json/1323608.json
fileName: Data/2022_female_json/1323609.json
fileName: Data/2022_female_json/1328485.json
fileName: Data/2022_female_json/1328486.json
fileName: Data/2022_female_json/1328487.json
fileName: Data/2022_female_json/1329247.json
fileName: Data/2022_female_json/1336981.json
fileName: Data/2022_female_json/1336982.json
fileName: Data/2022_female_json/1339555.json
fileName: Data/2022_female_json/1339558.json
fileName: Data/2022_female_json/1341226.json
fileName: Data/2022_female_json/1341227.json
fileName: Data/2022_female_json/1341228.json
fileName: 

In [10]:
df = pd.DataFrame(columns=['start_date', 'end_date', 'city', 'venue',
                          'event_name', 'match_type', 'match_number',
                          'team_type', 'team_1', 'team_2', 'toss_winner',
                          'toss_decision', 'innings', 'batting_team', 
                           'bowling_team', 'overs', 'striker', 'non_striker',
                          'bowler', 'singles', '4s', '6s', 'legbyes', 'byes',
                          'wides', 'no_balls', 'total_runs', 'wickets', 'wicket_type',
                          'wicket_taken_by', 'player_disissmied'])

In [11]:
df['start_date'] = start_date
df['end_date'] = end_date
df['city'] = city
df['venue'] = venue
df['event_name'] = event_name
df['match_type'] = match_type
df['match_number'] = match_number
df['team_type'] = team_type
df['team_1'] = team_1
df['team_2'] = team_2
df['toss_winner'] = toss_winner
df['toss_decision'] = toss_decision
df['innings'] = innings
df['batting_team'] = batting_team
df['bowling_team'] = bowling_team
df['overs'] = overs
df['striker'] = striker
df['non_striker'] = non_striker
df['bowler'] = bowler
df['singles'] = singles
df['4s'] = s4
df['6s'] = s6
df['legbyes'] = legbyes
df['byes'] = byes
df['wides'] = wides
df['no_balls'] = no_balls
df['total_runs'] = total_runs
df['wickets'] = wickets
df['wicket_type'] = wicket_type
df['wicket_taken_by'] = wicket_taken_by
df['player_disissmied'] = player_disissmied

In [12]:
df.set_index('start_date')
df.to_csv('Clean/woman/Ball_to_Ball/2022_female.csv', index=False)

In [13]:
df2 = pd.read_csv('Clean/woman/Ball_to_Ball/2022_female.csv', dtype={'match_number': 'str'})
df2.isna().sum()

start_date           0
end_date             0
city                 0
venue                0
event_name           0
match_type           0
match_number         0
team_type            0
team_1               0
team_2               0
toss_winner          0
toss_decision        0
innings              0
batting_team         0
bowling_team         0
overs                0
striker              0
non_striker          0
bowler               0
singles              0
4s                   0
6s                   0
legbyes              0
byes                 0
wides                0
no_balls             0
total_runs           0
wickets              0
wicket_type          0
wicket_taken_by      0
player_disissmied    0
dtype: int64

In [14]:
df2[300:350]

,start_date,end_date,city,venue,event_name,match_type,match_number,team_type,team_1,team_2,toss_winner,toss_decision,innings,batting_team,bowling_team,overs,striker,non_striker,bowler,singles,4s,6s,legbyes,byes,wides,no_balls,total_runs,wickets,wicket_type,wicket_taken_by,player_disissmied
300,2022-03-04,2022-03-04,Mount Maunganui,"Bay Oval, Mount Maunganui",ICC Women's World Cup,ODI,1244,international,West Indies,New Zealand,New Zealand,field,1,West Indies,New Zealand,48.5,SS Connell,A Mohammed,JM Kerr,1,0,0,0,0,0,0,1,0,Not Out,No Name,No Name
301,2022-03-04,2022-03-04,Mount Maunganui,"Bay Oval, Mount Maunganui",ICC Women's World Cup,ODI,1244,international,West Indies,New Zealand,New Zealand,field,1,West Indies,New Zealand,48.6,A Mohammed,SS Connell,JM Kerr,1,0,0,0,0,0,0,1,0,Not Out,No Name,No Name
302,2022-03-04,2022-03-04,Mount Maunganui,"Bay Oval, Mount Maunganui",ICC Women's World Cup,ODI,1244,international,West Indies,New Zealand,New Zealand,field,1,West Indies,New Zealand,49.1,A Mohammed,SS Connell,HM Rowe,0,0,0,0,0,0,0,0,0,Not Out,No Name,No Name
303,2022-03-04,2022-03-04,Mount Maunganui,"Bay Oval, Mount Maunganui",ICC Women's World Cup,ODI,1244,international,West Indies,New Zealand,New Zealand,field,1,West Indies,New Zealand,49.2,A Mohammed,SS Connell,HM Rowe,1,0,0,0,0,0,0,1,0,Not Out,No Name,No Name
304,2022-03-04,2022-03-04,Mount Maunganui,"Bay Oval, Mount Maunganui",ICC Women's World Cup,ODI,1244,international,West Indies,New Zealand,New Zealand,field,1,West Indies,New Zealand,49.3,SS Connell,A Mohammed,HM Rowe,1,0,0,0,0,0,0,1,0,Not Out,No Name,No Name
305,2022-03-04,2022-03-04,Mount Maunganui,"Bay Oval, Mount Maunganui",ICC Women's World Cup,ODI,1244,international,West Indies,New Zealand,New Zealand,field,1,West Indies,New Zealand,49.4,A Mohammed,SS Connell,HM Rowe,0,1,0,0,0,0,0,4,0,Not Out,No Name,No Name
306,2022-03-04,2022-03-04,Mount Maunganui,"Bay Oval, Mount Maunganui",ICC Women's World Cup,ODI,1244,international,West Indies,New Zealand,New Zealand,field,1,West Indies,New Zealand,49.5,A Mohammed,SS Connell,HM Rowe,0,1,0,0,0,0,0,4,0,Not Out,No Name,No Name
307,2022-03-04,2022-03-04,Mount Maunganui,"Bay Oval, Mount Maunganui",ICC Women's World Cup,ODI,1244,international,West Indies,New Zealand,New Zealand,field,1,West Indies,New Zealand,49.6,A Mohammed,SS Connell,HM Rowe,1,0,0,0,0,0,0,1,1,run out,AC Kerr,A Mohammed
308,2022-03-04,2022-03-04,Mount Maunganui,"Bay Oval, Mount Maunganui",ICC Women's World Cup,ODI,1244,international,West Indies,New Zealand,New Zealand,field,2,New Zealand,West Indies,0.1,SW Bates,SFM Devine,SS Connell,0,0,0,0,0,0,1,1,0,Not Out,No Name,No Name
309,2022-03-04,2022-03-04,Mount Maunganui,"Bay Oval, Mount Maunganui",ICC Women's World Cup,ODI,1244,international,West Indies,New Zealand,New Zealand,field,2,New Zealand,West Indies,0.2,SW Bates,SFM Devine,SS Connell,0,0,0,0,0,1,0,1,0,Not Out,No Name,No Name
